In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
#read the related data 
df= pd.read_csv("xxxxx.csv",parse_dates = ["date"])
df.head()

In [ ]:
#Fill NA values with 0 
df = df.fillna(0)

In [ ]:
#set date column as datetime
df["date"] = pd.to_datetime(df["date"])
df.info()

In [ ]:
#convert all possible dtypes into float or integer if any other exists
df["xxxxx"] = df["xxxxx"].astype(int)
df.info()

In [ ]:
#drop unrelated columns or columns include dtype object
df.drop(columns=["xxxx", "xxxx", "xxxx"], inplace = True, axis=1)

In [ ]:
#define one product df if there exists more than one product
product_id = 0
product_df = df[df["product_id"] == product_id].set_index("date")

In [ ]:
#Define targets and features
target = "demand"
features = ["xxxx", "xxxx", "xxxx"]

In [ ]:
#Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(product_df[features])
y_scaled = scaler_y.fit_transform(product_df[[target]])

In [ ]:
#def create_sequences(X, y, seq_length=14, horizon=7):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length:i+seq_length+horizon])
    return np.array(X_seq), np.array(y_seq)

SEQ_LENGTH = 14  # look-back 14 days
HORIZON = 7      # predict next 7 days

X_seq, y_seq = create_sequences(X_scaled, y_scaled, SEQ_LENGTH, HORIZON)

In [ ]:
# TRAIN/TEST SPLIT
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

In [ ]:
#construct the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(64, activation='relu', input_shape=(SEQ_LENGTH, X_train.shape[2])))
model_lstm.add(Dense(HORIZON))
model_lstm.compile(optimizer='adam', loss='mse')

In [ ]:
# TRAIN MODEL
history = model_lstm.fit(X_train, y_train, epochs=50, batch_size=16,
                    validation_split=0.2, verbose=1)

In [ ]:
#prediction
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, HORIZON))

y_test_actual = scaler_y.inverse_transform(y_test.reshape(-1, HORIZON))

In [ ]:
# Compute RMSE per horizon step
for i in range(HORIZON):
    rmse = np.sqrt(mean_squared_error(y_test_actual[:, i], y_pred[:, i]))
    df_compare = pd.DataFrame({
        "Actual": y_test_actual[:, i],
        "Predicted": y_pred[:, i]
    })
    print(f"\nHorizon t+{i+1} | RMSE: {rmse:.2f}")
    print(df_compare.head(10))

In [ ]:
#visualize the results
plt.figure(figsize=(14,6))
plt.plot(y_test_actual[:,0], label='Actual t+1')
plt.plot(y_pred[:,0], label='Predicted t+1')
plt.xlabel('Samples')
plt.ylabel('Demand')
plt.title('LSTM Forecast - Day 1 of 7-step horizon')
plt.legend()
plt.show()